# index进阶操作
下面介绍的方法只支持部分Index类型。

## 从index中恢复出原始数据
给定id，可以使用reconstruct或者reconstruct_n方法从index中回复出原始向量。  
支持IndexFlat, IndexIVFFlat (需要与make_direct_map结合), IndexIVFPQ, IndexPreTransform这几类索引类型。

In [1]:
# 导入faiss
import sys
import numpy as np 
import faiss

#生成数据
d = 16
n_data = 500
data = np.random.rand(n_data, d).astype('float32')

In [9]:
index = faiss.IndexFlatL2(d)
index.add(data)
re_data = index.reconstruct(0)  #指定需要恢复的向量的id,每次只能恢复一个向量
print(re_data)
re_data_n = index.reconstruct_n(0, 10) #从第0个向量开始，连续取10个
print(re_data_n.shape)

[0.58085376 0.5048806  0.99052334 0.5899147  0.5211166  0.35997516
 0.7275415  0.1242122  0.08336558 0.48458952 0.3289773  0.905333
 0.6513156  0.33422878 0.04078896 0.6842935 ]
(10, 16)


## 从index中移除向量
使用remove_ids方法可以移除Index中的部分向量，调用了IDSelector对象（或IDSelectorBatch批量操作）标识每个向量是否应该被移除。因为要遍历标识数据库中的每一个向量，所以只有在需要移除大部分向量时才建议使用。   
支持IndexFlat, IndexIVFFlat, IndexIVFPQ, IDMap。

In [2]:
np.arange(5)

array([0, 1, 2, 3, 4])

In [3]:
index = faiss.IndexFlatL2(d)
index.add(data)
print(index.ntotal)
index.remove_ids(np.arange(5)) # 需要移除的向量的id
print(index.ntotal)  #移除了5个向量，还剩495个

500
495


## 搜索距离范围内的向量
以查询向量为中心，返回距离在一定范围内的结果，如返回数据库中与查询向量距离小于0.3的结果。  
支持IndexFlat, IndexIVFFlat，只支持在CPU使用。

In [4]:
index = faiss.IndexFlatL2(d)
index.add(data)
dist = float(np.linalg.norm(data[3] - data[0])) * 0.99  # 定义一个半径/阈值
res_index = index.range_search(data[[49], :], dist)  #用第50个向量查询
print(res_index) #返回结果是一个三元组，分别是limit(返回的结果的数量), distance, index
res_index = index.range_search(data[[9], :], dist)  #用第10个向量查询
print(res_index) #返回结果是一个三元组，分别是limit(返回的结果的数量), distance, index

(array([ 0, 55]), array([1.280822  , 1.3295162 , 1.5801655 , 0.7207364 , 1.4991262 ,
       1.764689  , 0.        , 1.6234388 , 1.7517002 , 1.7792666 ,
       1.7567683 , 1.5430187 , 1.1690389 , 1.3218455 , 0.89941883,
       1.7498869 , 0.6897409 , 1.6804354 , 1.0435787 , 1.0258403 ,
       1.523227  , 1.4910573 , 1.1095209 , 1.7023015 , 1.3960974 ,
       1.3228252 , 1.6525512 , 1.8227777 , 1.4144393 , 1.6397526 ,
       1.2611463 , 1.7488983 , 1.308383  , 1.6685643 , 1.7254606 ,
       1.362961  , 1.47911   , 1.709446  , 1.6292217 , 1.7591525 ,
       1.5657973 , 1.6883831 , 1.4597831 , 1.3353482 , 1.8227344 ,
       1.5335355 , 1.6147892 , 1.6917641 , 1.045331  , 1.3066003 ,
       1.6605895 , 1.8068461 , 1.5812256 , 1.5959156 , 1.3412136 ],
      dtype=float32), array([  6,  24,  25,  31,  38,  44,  49,  53,  61,  67,  74,  83,  84,
        87,  99, 128, 133, 134, 146, 153, 154, 159, 179, 188, 192, 193,
       194, 196, 213, 233, 249, 258, 271, 275, 281, 289, 292, 310, 313,
      

## 拆分/合并index
可以将多个index合并，需要注意的是，多个Index的数据应该满足同一分布，并且用同一分布的数据训练index，如果多个Index的数据分布不同，合并时并不会报错，但在理论上会降低索引的精度，应该用与合并后的数据集同分布的训练集再次训练。

In [35]:
nlist = 10
quantizer = faiss.IndexFlatL2(d)
index1 = faiss.IndexIVFFlat(quantizer, d, nlist)
index1.train(data)
index1.add(data[:250])
index2 = faiss.IndexIVFFlat(quantizer, d, nlist)
index2.add(data[250:])
index1.merge_from(index2, 250)
print(index1.ntotal) # 合并后应该包含500个向量
dis, ind = index1.search(data[:5], 10)
print(ind)

500
[[  0  28 382 194 286 114 308 480 254 279]
 [  1 416 272 250 296 138 366 281  93 169]
 [  2  44 491 231 178 285 117 273  83 187]
 [  3 194  28 143 270 430 264 382 197 279]
 [  4 464 317  89 325 498  83 101 285  51]]
